In [1]:
using Pkg
Pkg.activate("../cheb_bessel_tests-main")

  Activating project at `~/Desktop/PhD_Stuff/cheb_bessel_tests-main`


In [2]:
using NPZ
using Interpolations
using DataInterpolations

[ Info: Precompiling DataInterpolations [82cc6244-b520-54b8-b5a6-8a565e85f1d0]
Precompiling DataInterpolationsChainRulesCoreExt
  ✓ DataInterpolations → DataInterpolationsChainRulesCoreExt
  1 dependency successfully precompiled in 3 seconds. 40 already precompiled.
[ Info: Precompiling DataInterpolationsChainRulesCoreExt [187dfacc-d000-5da0-94aa-5ec8eb045cfd]
┌ Warning: Module DataInterpolations with build ID fafbfcfd-afe2-4f7d-0001-000054e461f4 is missing from the cache.
│ This may mean DataInterpolations [82cc6244-b520-54b8-b5a6-8a565e85f1d0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1942
[ Info: Skipping precompilation since __precompile__(false). Importing DataInterpolationsChainRulesCoreExt [187dfacc-d000-5da0-94aa-5ec8eb045cfd].


In [3]:
output_dir = "FKEM/inputs/"

"FKEM/inputs/"

## Step 1: Power Spectrum

In [4]:
k = npzread("data/k_nl.npz")["arr_0"]
z = npzread("data/z_nl.npz")["arr_0"]
pk_lin = npzread("data/camb_lin_pk.npz")["arr_0"]
pk_nl = npzread("data/camb_nl_pk.npz")["arr_0"];

In [5]:
data_dict = Dict(
    "k" => k,
    "z" => z,
    "pk_lin" => pk_lin,
    "pk_nl" => pk_nl
)
npzwrite(output_dir*"pk.npz", data_dict)

## Background quantities

In [6]:
using Blast

Precompiling Blast
  ✓ Blast
  1 dependency successfully precompiled in 16 seconds. 338 already precompiled.
[ Info: Precompiling Blast [aee4e916-1e40-4aff-8d35-b84b50e5d5e3]
┌ Warning: Module DataInterpolations with build ID fafbfcfd-afe2-4f7d-0001-000054e461f4 is missing from the cache.
│ This may mean DataInterpolations [82cc6244-b520-54b8-b5a6-8a565e85f1d0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1942
[ Info: Skipping precompilation since __precompile__(false). Importing Blast [aee4e916-1e40-4aff-8d35-b84b50e5d5e3].


In [7]:
z = npzread("data/mynz.npy")["z"]
cosmo = Blast.FlatΛCDM()
grid = Blast.CosmologicalGrid(z_range=z)
bg = Blast.BackgroundQuantities(Hz_array = zeros(length(z)), χz_array=zeros(length(z)))
Blast.evaluate_background_quantities!(grid, bg, cosmo)

In [8]:
z_of_χ = DataInterpolations.AkimaInterpolation(grid.z_range, bg.χz_array, extrapolate=true);

In [9]:
Ez = bg.Hz_array/cosmo.H0
chi = bg.χz_array

data_dict = Dict(
    "chi" => chi,
    "z" => z,
    "Ez" => Ez
)
npzwrite(output_dir*"background.npz", data_dict)

## Kernels

In [10]:
kernel_cl = npzread("data/ccl_gal_kernel_finer.npz")["arr_0"]
kernel_sh = npzread("data/ccl_sh_kernel_finer.npz")["arr_0"]
kernel_cmb = npzread("data/ccl_cmb_kernel_finer.npz")["arr_0"]
chi_sh = npzread("data/chi_finer.npz")["arr_0"]
z_sh = z_of_χ.(chi_sh)
kernel_cl = reshape(kernel_cl, 1, 2000)
kernel_sh = reshape(kernel_sh, 1, 2000)
kernel_cmb = reshape(kernel_cmb, 1, 2000)

1×2000 Matrix{Float64}:
 0.0  9.54626e-8  1.9104e-7  2.86731e-7  …  0.000491187  0.000491713

In [11]:
data_dict = Dict(
    "chi_cl" => chi_sh,
    "z_cl" => z_sh,
    "chi_sh" => chi_sh,
    "z_sh" => z_sh,
    "kernels_cl" => kernel_cl,
    "kernels_sh" => kernel_sh,
    "kernels_cmb" => kernel_cmb
)
npzwrite(output_dir*"kernels.npz", data_dict)
